In [9]:
#Sort Different columns: 
import pandas as pd 
import numpy as np
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit_aer.primitives import Sampler
import re
import openpyxl

df = pd.read_csv('Files/student_dataset.csv')
type = detectColumns(df,df.columns)
print(type)


    

{'Student_Names': {'1': 59, '0': 40}, 'Phone_No.': {'0': 79, '1': 20}, 'Math': {'0': 79, '1': 20}, 'Physics': {'0': 80, '1': 19}, 'Chemistry': {'1': 20, '0': 79}, 'Grade': {'0': 89, '1': 10}, 'Comment': {'0': 40, '1': 59}, 'Roll No.': {'0': 69, '1': 30}, 'School Name': {'1': 59, '0': 40}, 'Student Address': {'1': 60, '0': 39}}


In [20]:
def dataClean(df,colTypes):
    for i in df.index[(df.isna().sum(axis=1)==len(df.columns))]:
        df.drop(index=i, inplace=True)
        
    for i in colTypes.keys():
        if df[i].isna().sum() >=len(df[i])-2:
            df.drop(columns=[i],inplace=True)
            continue
        
        if df[i].isna().sum() != 0:#IF column has no type
                print(i)
                if (colTypes[i]['0'] == 60 or colTypes[i]['1'] == 40): # if column is type of year only
                    df[i] = df[i].fillna(2100)
                elif colTypes[i]['0'] == 30 or  colTypes[i]['1'] == 70:#if column is type of date only 
                    df[i] = df[i].fillna(pd.to_datetime('2030-01-01'))
                elif colTypes[i]['0'] == 20 or  colTypes[i]['1'] == 80 :#if column is type of date and time 
                    df[i]= df[i].fillna('2030-01-01')
                elif colTypes[i]['0'] == 70 or colTypes[i]['1']==30: #Roll no type
                    df[i] = df[i].fillna(000000)
                elif colTypes[i]['1']==50 or colTypes[i]['0']==50:#if it is of type id 
                    df[i]=df[i].fillna("Unknown")
                elif colTypes[i]['0']==90 or colTypes[i]['1'] ==10:#if it of type oneor2digit
                    df[i] = df[i].fillna(0)
                elif colTypes[i]['0'] == 80 or  colTypes[i]['1'] == 20 : #if it type of nemric
                    df[i]=df[i].fillna(0.0)
                elif colTypes[i]['0'] == 40 or  colTypes[i]['1'] == 60 : #string or object
                    df[i] = df[i].fillna('NULL-NAN')
        continue
    return df

df = pd.read_csv('Files/twitter_training.csv')
col = detectColumns(df,df.columns)
print(df.isna().sum())
df = dataClean(df,col)
print(df.isna().sum())


Num          0
company      0
Type         0
Comments    41
dtype: int64
Comments
Num         0
company     0
Type        0
Comments    0
dtype: int64


In [6]:
def FindDuplicateInStrOrOBj(series):
    series = pd.Series(series)
    series = series[series.duplicated()==True]
    return series

df = pd.read_csv('Files/twitter_training.csv')
dup = FindDuplicateInStrOrOBj(df['Comments'])
print(dup)
print(len(df))
df.drop_duplicates(inplace=True,subset=['Comments'])
print(len(df))

13      Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...
20      that was the first borderlands session in a lo...
26      The biggest disappointment of my life came a y...
51      Blaming Sight for Tardiness! A little bit of b...
64                                                   .. [
                              ...                        
4754    At the same time, despite the fact that there ...
4755                                                  NaN
4756                                                  NaN
4757                                                  NaN
4796                               buff.ly / 33Q7U7s.....
Name: Comments, Length: 288, dtype: object
4800
4512


In [5]:
df = pd.read_csv('Files/employee_time_log.csv') 
df = kmeansOnString(df,'Employee_ID')
print(df)

time By Model  14.663967370986938
Time in Normalizig : 0.0017213821411132812
Time clustering : 0.7560210227966309
Silhouette Score: 0.08340984
Time in Normalizig : 0.04461359977722168
    Employee_ID Task_ID  Start_Date  End_Date  login Time Logout_Time
985       E9828  T17194  25-01-2025       NaN         NaN    18:31:11
999       E9115  T37857  30-01-2025       NaN         NaN    19:06:08
572       E5099  T11336  10-01-2025       NaN         NaN    18:16:00
573       E8987  T68207  26-01-2025       NaN         NaN    18:29:35
574       E9529  T85829  23-01-2025       NaN         NaN    18:39:01
..          ...     ...         ...       ...         ...         ...
969       E6790  T23862  23-12-2024       NaN         NaN    17:39:57
37        E2356  T35740  04-01-2025       NaN         NaN    19:40:28
34        E6234  T92956  15-01-2025       NaN         NaN    20:28:31
31        E5396  T59696  05-12-2024       NaN         NaN    18:00:33
25        E9638  T56134  25-12-2024       NaN 

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd
import time
def kmeansOnString(df,col):
    phrase = (df[col]).astype(str)
    # Step 1: Convert strings to embeddings using a Sentence Transformer (BERT-based model)
    st = time.time()
    model = SentenceTransformer('all-MiniLM-L6-v2')  # a lightweight, efficient model
    embeddings = model.encode(phrase)
    et = time.time()
    print("time By Model ", et-st)
    
    st = et
    # Optionally, normalize embeddings if needed:
    from sklearn.preprocessing import normalize
    embeddings = normalize(embeddings)
    et = time.time()
    print(f"Time in Normalizig : {et-st}")
    
    st = et
    # Step 2: Apply KMeans clustering on these embeddings
    # Using n_init and max_iter helps ensure stable convergence.
    k =  int(np.ceil(len(df[col]) / 80))# number of clusters
    kmeans = KMeans(n_clusters=7, n_init=19, max_iter=450, random_state=42)
    df['Cluster Id'] = kmeans.fit_predict(embeddings)
    et = time.time()
    print(f"Time clustering : {et-st}")
    
    st = et
    # Step 3: Evaluate the clustering with Silhouette Score
    score = silhouette_score(embeddings, df['Cluster Id'])
    print("Silhouette Score:", score)
    et = time.time()
    print(f"Time in Normalizig : {et-st}")
    
    df = df.sort_values(by='Cluster Id').drop(columns=['Cluster Id'])
    return df

c:\Users\tikes\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from qiskit import QuantumCircuit
import pandas as pd
import math
def detectColumns(df, prioColumns):
    result = {}    
    for col in prioColumns:
        qc =  QuantumCircuit(1,1)
        col_data = df[col]
        col_str = df[col].astype(str).str.strip()
        # 3. Check for Date values (type 2)
        if check_date_format(col_str):
            p = 0.70000
                
        # 4. Check for DateTime values (type 3)
        elif check_datetime_format(col_str):
            p = 0.800000
            
        elif OneOr2digitDetection(col_data):
            p = 0.100000
        # 1. Check for Roll Numbers (type 4)
        elif pd.api.types.is_numeric_dtype(col_data):
            p = 0.2000
            # 2. Check for Year values (type 1)
            if check_year_values(col_data):
                p = 0.40000
            elif check_roll_number(col_data):
                p = 0.30000

        elif pd.api.types.is_string_dtype(df[col]) or pd.api.types.is_object_dtype(df[col]):
            p = 0.600000
            if detectIdTypeCol(col_data):
                p = 0.5
        
        angle = 2 * math.asin(math.sqrt(p))
        qc.ry(angle, 0)
        # Initialize result as 0 (unrecognized type)
        result[col] = measurCir(qc,0)
        
    return result

def check_roll_number(col_data):
    try:
        # Convert numbers to strings for checking patterns
        sample_start = [str(i) for i in col_data.head(10)]
        sample_middle = [str(i) for i in col_data.iloc[int(len(col_data)/2)-5:int(len(col_data)/2)+5]]
        sample_end = [str(i) for i in col_data.iloc[-10:]]
        
        # Combine samples
        samples = sample_start + sample_middle + sample_end
        
        # Check if all numbers have the same length and >= 5 digits
        if len(set(len(str(x)) for x in samples)) == 1:
            length = len(str(samples[0]))
            if length >= 5:
                # Check if all numbers start with the same digit
                first_digit = str(samples[0])[0]
                return all(str(x).startswith(first_digit) for x in samples)
    except:
        pass
    return False

def check_year_values(col_data):
    if pd.api.types.is_string_dtype(col_data):
        try:
            col_data = pd.to_numeric(col_data)
        except:
            pass
    # Handle if column is numeric and looks like a year
    if pd.api.types.is_numeric_dtype(col_data) or  pd.api.types.is_float_dtype(col_data) or  pd.api.types.is_integer_dtype(col_data):
        if col_data.dropna().empty == False:
            if pd.api.types.is_float_dtype(col_data):
            # Check if float values have only 2 decimal places
                if col_data.dropna().apply(lambda x: round(x, 2) == x).all():
                    if col_data.dropna().between(1800, 2050).all():
                        return True # Only year
            # For integer values
            elif col_data.dropna().between(1800, 2100).all():
                True # Only year
                
    return False

def check_date_format(col_str):
    # Check for common date formats (yyyy-mm-dd, dd-mm-yyyy, etc.)
    date_pattern = r'^\d{1,4}[-/\.]\d{1,2}[-/\.]\d{1,4}$'
    return col_str.str.match(date_pattern).all()

def check_datetime_format(col_str):
    # Check for datetime format (date + time)
    datetime_pattern = r'\d{1,4}[-/\.]\d{1,2}[-/\.]\d{1,4}.*\d{1,2}:\d{2}'
    return col_str.str.contains(datetime_pattern).all()

def detectIdTypeCol(col_data):
    if pd.api.types.is_string_dtype(col_data):
        pattern = r'\b[A-Z0-9]{1,4}[-_./]?[A-Z0-9]{2,6}[-_./]?[A-Z0-9]{0,5}\b'
        return all(re.fullmatch(pattern, item) for item in col_data)
    
def OneOr2digitDetection(col_data):
    non2Digit=0
    for v  in col_data:
        if len(str(v)) >= 2:
            non2Digit=o=non2Digit+1
    
    if len(col_data)/2.2 > non2Digit:
        return True
    return False

def detectSimpleDtypes(col_data):
    if pd.api.types.is_integer_dtype(col_data):
        return 'allInt'
    if pd.api.types.is_float_dtype(col_data):
        if col_data.isna().all()== False:
            return 'numaric'
    if pd.api.types.is_string_dtype(col_data) or pd.api.types.is_object_dtype(col_data):
        return 'str'
    return None

def clusterDateTimeCol(fContent, col,no,ascending=True):
    if no ==1:
        # Try to detect and sort if the column is just year values
        fContent = fContent.sort_values(by=col,ignore_index=True, ascending=ascending)
        fContent = fContent.reset_index(drop=True)
    elif no == 2 or 3:
        # Now, try to detect proper date/datetime columns
        try:
            # Avoid processing numeric-only or zero-filled columns
            sample_vals = fContent[col].astype(str).str.strip().replace('0', np.nan).dropna()
            if len(sample_vals) == 0:
                return fContent# All values are zero or empty-like
            
            # Try parsing
            try:
                parsed_col = pd.to_datetime(fContent[col], errors='raise')
            except:
                parsed_col = pd.to_datetime(fContent[col], dayfirst=True, errors='raise')
               
            if all(parsed_col.dt.time == pd.to_datetime('00:00:00').time()) and no == 2:  # Only date
                fContent[col] = parsed_col
                fContent = clean_and_sort_date_column(fContent, col, ascending)
                fContent = fContent.reset_index(drop=True)
                # Replace original column with parsed datetime values
            elif no == 3:  # Date + time
                fContent[col] = parsed_col
                fContent = handle_datetime_column(fContent, col, ascending)
                fContent = fContent.reset_index(drop=True)
        except Exception as e:
            return fContent # Not a datetime column
        
    return fContent 

def clean_and_sort_date_column(dff, column_name, ascending=True):
        try:
            
            # Step 2: Drop NaT (invalid formats)
            dff = dff.dropna(subset=[column_name])
            
            # Step 3: Sort the DataFrame by that column
            dff = dff.sort_values(by=column_name, ascending=ascending)

            # Optional: Format to clean date string (YYYY-MM-DD)
            dff[column_name] = dff[column_name].dt.strftime('%Y-%m-%d')
            print(f"{column_name} date called ")
            return dff
        
        except Exception as e:
            print(f"⚠️ Error while processing date column: {e}")
            return dff

def handle_datetime_column(df, column_name, ascending):
    print(f"{column_name} dateTime")
    # Check if most values in column are datetime with time
    values = df[column_name].dropna().astype(str).head(20)
    count_datetime = sum([pd.api.is_datetime(v) for v in values])

    if count_datetime >= len(values) // 2:  # At least half must be datetime-like
        # Convert full column to datetime
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
        # Drop rows with invalid dates
        df = df.dropna(subset=[column_name])
        # Sort by that column
        df = df.sort_values(by=column_name,ascending=ascending).reset_index(drop=True)
        print(f"[INFO] '{column_name}' successfully recognized and sorted as datetime.")
    else:
        print(f"[INFO] '{column_name}' does not contain proper datetime with time.")

    return df

In [2]:
def measurCir(qc,q_num):
    qc.measure(q_num,q_num)
    simulator = AerSimulator()
    # Transpile & run
    compiled = transpile(qc, simulator)
    r = simulator.run(compiled, shots=100000).result()
    counts = r.get_counts()
    for k,v in counts.items():
        counts[k] = int(v/1000)
    return counts